In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, random_split

In [2]:
# Set random seed
torch.manual_seed(42)
np.random.seed(42)

# Generate data
X = np.random.rand(100, 2)
y = (X[:, 0] + X[:, 1] > 1).astype(np.int64)
X_tensor = torch.FloatTensor(X)
y_tensor = torch.LongTensor(y)

# Create dataset and split
dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [3]:
# Define neural network
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x

# Initialize model, loss, and optimizer
model = SimpleNN(input_size=2, hidden_size=10, output_size=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [4]:
# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    for X_batch, y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * X_batch.size(0)
    
    train_loss /= len(train_loader.dataset)
    
    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item() * X_batch.size(0)
            predicted = torch.argmax(outputs, dim=1)
            val_correct += (predicted == y_batch).sum().item()
    
    val_loss /= len(val_loader.dataset)
    val_accuracy = val_correct / len(val_loader.dataset)
    
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {train_loss:.4f}, '
              f'Val Loss: {val_loss:.4f}, '
              f'Val Accuracy: {val_accuracy:.4f}')

# Final evaluation
model.eval()
val_correct = 0
with torch.no_grad():
    for X_batch, y_batch in val_loader:
        outputs = model(X_batch)
        predicted = torch.argmax(outputs, dim=1)
        val_correct += (predicted == y_batch).sum().item()
val_accuracy = val_correct / len(val_loader.dataset)
print(f'Final Validation Accuracy: {val_accuracy:.4f}')

Epoch [20/100], Train Loss: 0.7054, Val Loss: 0.7292, Val Accuracy: 0.3000
Epoch [40/100], Train Loss: 0.6912, Val Loss: 0.6911, Val Accuracy: 0.5500
Epoch [60/100], Train Loss: 0.6807, Val Loss: 0.6651, Val Accuracy: 0.7000
Epoch [80/100], Train Loss: 0.6700, Val Loss: 0.6425, Val Accuracy: 0.7000
Epoch [100/100], Train Loss: 0.6592, Val Loss: 0.6226, Val Accuracy: 0.7000
Final Validation Accuracy: 0.7000
